In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import tsdm

In [ ]:
ds = tsdm.datasets.KIWI_RUNS()

In [ ]:
ds.timeseries